In [15]:
import findspark
findspark.init()
import os
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
sc = (
    SparkSession.builder
    .appName("vis")
    .master("local")
    .enableHiveSupport()
    .getOrCreate()
)

23/01/08 18:12:37 WARN util.Utils: Your hostname, node1 resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
23/01/08 18:12:37 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/01/08 18:12:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/08 18:12:40 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/08 18:12:40 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [45]:
month_temp_stats = sc.sql("""
    select
        Rok,
        `Miesiąc`,
        avg(`Średnia temperatura dobowa [°C]`) as avg_temp,
        avg(`Maksymalna temperatura dobowa [°C]`) as max_temp,
        avg(`Minimalna temperatura dobowa [°C]`) as min_temp
    from klimat group by Rok, `Miesiąc`
""").toPandas()

In [46]:
month_temp_stats.to_csv('stats/month_temp.csv')

In [36]:
month_rain_stats = sc.sql("""
    select
        Rok,
        `Miesiąc`,
        sum(`Suma dobowa opadów [mm]`) as sum_rain
    from klimat group by Rok, `Miesiąc`
""").toPandas()

In [37]:
month_rain_stats.to_csv('stats/month_rain.csv')

In [41]:
station_temp_stats = sc.sql("""
    select
        `Nazwa stacji`,
        avg(`Średnia temperatura dobowa [°C]`) as avg_temp
    from klimat
    where Rok > 2010
    group by `Nazwa stacji`
""").toPandas()

In [42]:
station_temp_stats.to_csv('stats/station_temp.csv')

In [100]:
month_synop = sc.sql("""
    select
        Rok,
        `Miesiąc`,
        sum(`Czas trwania opadu deszczu [godziny]`) / sum(case when `Status pomiaru DESZ` = 'brak pomiaru' then 0 else 1 end) as avg_time_deszcz,
        sum(`Czas trwania opadu śniegu [godziny]`) / sum(case when `Status pomiaru SNEG` = 'brak pomiaru' then 0 else 1 end) as avg_time_snieg,
        sum(`Czas trwania opadu deszczu ze śniegiem [godziny]`) / sum(case when `Status pomiaru DISN` = 'brak pomiaru' then 0 else 1 end) as avg_time_deszcz_ze_sniegiem,
        sum(`Czas trwania gradu [godziny]`) / sum(case when `Status pomiaru GRAD` = 'brak pomiaru' then 0 else 1 end) as avg_time_grad,
        sum(`Czas trwania mgły [godziny]`) / sum(case when `Status pomiaru MGLA` = 'brak pomiaru' then 0 else 1 end) as avg_time_mgla,
        sum(`Czas trwania zamglenia`) / sum(case when `Status pomiaru ZMGL` = 'brak pomiaru' then 0 else 1 end) as avg_time_zamglenie,
        sum(`Czas trwania sadzi [godziny]`) / sum(case when `Status pomiaru SADZ` = 'brak pomiaru' then 0 else 1 end) as avg_time_sadz,
        sum(`Czas trwania gołoledzi [godziny]`) / sum(case when `Status pomiaru GOLO` = 'brak pomiaru' then 0 else 1 end) as avg_time_gololedz,
        sum(`Czas trwania zamieci śnieżnej niskiej [godziny]`) / sum(case when `Status pomiaru ZMNI` = 'brak pomiaru' then 0 else 1 end) as avg_time_zamiec_niska,
        sum(`Czas trwania zamieci śnieżnej wysokiej [godziny]`) / sum(case when `Status pomiaru ZMWS` = 'brak pomiaru' then 0 else 1 end) as avg_time_zamiec_wysoka,
        sum(`Czas trwania zmętnienia [godziny]`) / sum(case when `Status pomiaru ZMET` = 'brak pomiaru' then 0 else 1 end) as avg_time_zmetnienie,
        sum(`Czas trwania wiatru >=10m/s [godziny]`) / sum(case when `Status pomiaru FF10` = 'brak pomiaru' then 0 else 1 end) as avg_time_wiatr_10ms,
        sum(`Czas trwania wiatru >15m/s [godziny]`) / sum(case when `Status pomiaru FF15` = 'brak pomiaru' then 0 else 1 end) as avg_time_wiatr_15ms,
        sum(`Czas trwania burzy`) / sum(case when `Status pomiaru BRZA` = 'brak pomiaru' then 0 else 1 end) as avg_time_burza,
        sum(`Czas trwania rosy`) / sum(case when `Status pomiaru ROSA` = 'brak pomiaru' then 0 else 1 end) as avg_time_rosa,
        sum(`Czas trwania szronu [godziny]`) / sum(case when `Status pomiaru SZRO` = 'brak pomiaru' then 0 else 1 end) as avg_time_szron
    from synop_test
    group by Rok, `Miesiąc`
""").toPandas()

In [101]:
month_synop.to_csv('stats/month_synop.csv')